# Launch a new instance of ASReview

Make sure to add your own authenication token from ngrok.com to the cell below.

ASReview can be opened using the link found in the output of the cell near the
end of the notebook.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JTeijema/ASReview-Google-Colab/blob/main/ASReview_in_colab.ipynb)

In [ ]:
# installing packages

!pip install pyngrok --quiet
!pip install asreview --quiet

In [ ]:
# imports
from pyngrok import ngrok

In [ ]:
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "your_token"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# Terminate open tunnels if exist
ngrok.kill()

In [ ]:
# Connect to the tunnel
tunnel = ngrok.connect(port="80", proto="http")

Below are links to the new instance of ASReview.

In [ ]:
ngrok.get_tunnels()

In [ ]:
!asreview lab --port 80